<a href="https://colab.research.google.com/github/Apolo4427/-inteligencia_artificial/blob/main/02_Modelos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [61]:
# Bibliotecas utilizadas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import OneHotEncoder
import warnings
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor

from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline

# Carga de datos
data_train = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vQHU9r-osSfdm1EQ1qT4SqM13kdFN5ELQStnSx-n4jOzSr4jxrg-YMG4bCe9W5Kkw1qelHxSKrfZlMi/pub?gid=1834913949&single=true&output=csv')
data_test = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vSOz7KZRcRa_l7f4QF5wk5W5LXvAZZBMgr5hdVGt2d_Pxm8n4fqnPQ4n6dSLv0H2nkNx19SKUzGZhX4/pub?gid=854072752&single=true&output=csv')


In [59]:
# nombres de las columnas test
column_names = data_test.columns

print(column_names)


Index(['budget', 'genres', 'original_language', 'popularity',
       'production_companies', 'production_countries', 'release_date',
       'runtime', 'spoken_languages', 'cast'],
      dtype='object')


# Modelos
###Modelo con Random Forest

para este primer acercamiento usaremos nos enfocaremos en las variables de ('budget', 'popularity', 'runtime') puesto que consideramos que son los factores mas influyentes.

In [60]:
# columnas
columnas = ['budget', 'popularity', 'runtime']
X_train = data_train[columnas]
y_train = data_train['revenue']

X_test = data_test[columnas]

# Filtrar valores nulos
X_train = X_train.fillna(0)
y_train = y_train.fillna(0)
X_test = X_test.fillna(0)

# Limpiar y transformar las columnas con formato incorrecto
for columna in columnas:
    X_train[columna] = pd.to_numeric(X_train[columna].astype(str).replace('[^\d.]', '', regex=True), errors='coerce')
    X_test[columna] = pd.to_numeric(X_test[columna].astype(str).replace('[^\d.]', '', regex=True), errors='coerce')

# Eliminar valores nulos en X_train y y_train
filas_nulas_entrenamiento = X_train[X_train.isnull().any(axis=1) | y_train.isnull()]
X_train = X_train.drop(filas_nulas_entrenamiento.index)
y_train = y_train.drop(filas_nulas_entrenamiento.index)

# Dividir datos de entrenamiento
X_train_entrenamiento, X_train_evaluacion, y_train_entrenamiento, y_train_evaluacion = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Crear modelo de bosque aleatorio
modelo = RandomForestRegressor(n_estimators=500, max_depth=10, random_state=42)

# Entrenar el modelo
modelo.fit(X_train_entrenamiento, y_train_entrenamiento)

# Realizar predicciones en el conjunto de evaluación
predicciones_evaluacion = modelo.predict(X_train_evaluacion)

# Error Cuadrático Medio (MSE)
mse = mean_squared_error(y_train_evaluacion, predicciones_evaluacion)
print(f'Error Cuadrático Medio en el conjunto de evaluación: {mse}')

# Error Porcentual Absoluto Medio (MAPE)
mape = mean_absolute_error(y_train_evaluacion, predicciones_evaluacion) / y_train_evaluacion.mean() * 100
print(f'Error Porcentual Absoluto Medio en el conjunto de evaluación: {mape}%')


Error Cuadrático Medio en el conjunto de evaluación: 8008336688225830.0
Error Porcentual Absoluto Medio en el conjunto de evaluación: 68.77933690407436%


Es evidente que el error es demasiado elevado, con locual concluimos que apesar de que parece que estas variables son las mas influyentes a prinmera vista, no son suficientemente descriptivas en un ambito general.

A partir de de este entendimiento, integraremos mas columnas al modelo, esperando que mejore la calidad de la predicción


In [53]:
# columnas
columnas_seleccionadas = ['budget', 'genres', 'original_language', 'popularity', 'production_companies',
                           'production_countries', 'release_date', 'runtime', 'spoken_languages', 'cast', 'revenue']

data_train = data_train[columnas_seleccionadas]
data_test = data_test[columnas_seleccionadas[:-1]]

# valores nulos
data_train = data_train.fillna(0)
data_test = data_test.fillna(0)

# Limpiar
columnas_numericas = ['budget', 'popularity', 'runtime']
for columna in columnas_numericas:
    data_train[columna] = pd.to_numeric(data_train[columna].astype(str).replace('[^\d.]', '', regex=True), errors='coerce')
    data_test[columna] = pd.to_numeric(data_test[columna].astype(str).replace('[^\d.]', '', regex=True), errors='coerce')

# columnas categóricas
columnas_categoricas = ['genres', 'original_language', 'production_companies', 'production_countries', 'spoken_languages', 'cast']
data_train[columnas_categoricas] = data_train[columnas_categoricas].astype(str)
data_test[columnas_categoricas] = data_test[columnas_categoricas].astype(str)

# Codificación one-hot para variables categóricas
with warnings.catch_warnings():
    warnings.simplefilter(action='ignore', category=UserWarning)
    encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')

    # codificación one-hot
    data_train_encoded = pd.DataFrame(encoder.fit_transform(data_train[columnas_categoricas]))
    data_train_encoded.columns = data_train_encoded.columns.astype(str)

    data_test_encoded = pd.DataFrame(encoder.transform(data_test[columnas_categoricas]))
    data_test_encoded.columns = data_test_encoded.columns.astype(str)

# Concatenar las columnas
data_train_final = pd.concat([data_train[columnas_numericas], data_train_encoded], axis=1)
data_test_final = pd.concat([data_test[columnas_numericas], data_test_encoded], axis=1)

# Dividir datos
X_train_entrenamiento, X_train_evaluacion, y_train_entrenamiento, y_train_evaluacion = train_test_split(data_train_final, data_train['revenue'], test_size=0.2, random_state=42)

# imputador
imputador = SimpleImputer(strategy='mean')

# imputación en los datos
X_train_entrenamiento_imputado = imputador.fit_transform(X_train_entrenamiento)
X_train_evaluacion_imputado = imputador.transform(X_train_evaluacion)
data_test_final_imputado = imputador.transform(data_test_final)

# Crear modelo
modelo = HistGradientBoostingRegressor(max_depth=10, random_state=42)

# Entrenar el modelo
modelo.fit(X_train_entrenamiento, y_train_entrenamiento)


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [58]:
# Realizar predicciones en el conjunto de evaluación
predicciones_evaluacion = modelo.predict(X_train_evaluacion)

# Calcular el Error Cuadrático Medio (MSE)
mse = mean_squared_error(y_train_evaluacion, predicciones_evaluacion)
print(f'Error Cuadrático Medio en el conjunto de evaluación: {mse}')

# Error Porcentual Absoluto Medio (MAPE)
mape = mean_absolute_error(y_train_evaluacion, predicciones_evaluacion) / y_train_evaluacion.mean() * 100
print(f'Error Porcentual Absoluto Medio en el conjunto de evaluación: {mape}%')

# Realizar predicciones en el conjunto de prueba
predicciones_test = modelo.predict(data_test_final)

print([round(prediccion, 2) for prediccion in predicciones_test])

Error Cuadrático Medio en el conjunto de evaluación: 7017777062295636.0
Error Porcentual Absoluto Medio en el conjunto de evaluación: 65.80428128788864%
[5152622.29, 17127512.59, 14684869.39, 24540302.99, 475746.99, 6405080.12, -4648307.05, 87309394.63, 65897190.85, 265541186.76, 2850370.9, -338638.27, 24241885.59, 7513195.7, 49913399.5, 14884125.27, 59468056.73, 170392061.63, 26256513.58, 785042696.86, 71526771.71, 40418351.74, 16096892.62, 44466113.65, 4226496.85, 110506264.77, 22002202.96, 183915390.5, 3821558.99, 55309696.01, 16048720.1, 21317455.94, 13790878.62, 11788928.31, 23223185.2, 20196721.47, 25715516.06, 18434308.15, 23863395.66, 13270093.76, 38152825.79, 12456811.95, 44114607.7, 5416265.73, 381217545.75, 11151288.02, 15715043.91, 75776441.67, 11000428.34, 12503083.41, 14534091.03, 63442005.78, 6508882.96, 5639046.83, 108227000.78, 18806855.98, 127348526.87, 854639137.63, -4255693.55, 128999772.74, 41083221.34, 16292750.88, 29232879.58, 297115746.72, 81653493.12, 186057376

Se puede apreciar un menor porcentaje de error, sin embargo este sigue siendo relativamente alto, esto puede deberse a la influencia presente en el resto de caracteristicas que no fueron consideradas en este modelo

###modelo con SVM

Probaremos el modelo SVM con la intencion de comparar resultados


In [62]:
#  valores nulos
data_train = data_train.fillna(0)
data_test = data_test.fillna(0)

# Limpiar
columnas_numericas = ['budget', 'popularity', 'runtime']
for columna in columnas_numericas:
    data_train[columna] = pd.to_numeric(data_train[columna].astype(str).replace('[^\d.]', '', regex=True), errors='coerce')
    data_test[columna] = pd.to_numeric(data_test[columna].astype(str).replace('[^\d.]', '', regex=True), errors='coerce')

# Convertir categóricas
columnas_categoricas = ['genres', 'original_language', 'production_companies', 'production_countries', 'spoken_languages', 'cast']
data_train[columnas_categoricas] = data_train[columnas_categoricas].astype(str)
data_test[columnas_categoricas] = data_test[columnas_categoricas].astype(str)

# Codificación one-hot
encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')

# Aplicar codificación one-hot
data_train_encoded = pd.DataFrame(encoder.fit_transform(data_train[columnas_categoricas]))
data_train_encoded.columns = data_train_encoded.columns.astype(str)

data_test_encoded = pd.DataFrame(encoder.transform(data_test[columnas_categoricas]))
data_test_encoded.columns = data_test_encoded.columns.astype(str)

# Concatenar las columnas
data_train_final = pd.concat([data_train[columnas_numericas], data_train_encoded], axis=1)
data_test_final = pd.concat([data_test[columnas_numericas], data_test_encoded], axis=1)

# Dividir datos
X_train_entrenamiento, X_train_evaluacion, y_train_entrenamiento, y_train_evaluacion = train_test_split(data_train_final, data_train['revenue'], test_size=0.2, random_state=42)

# Crear modelo de SVM con un pipeline que incluye imputación de valores nulos
modelo = make_pipeline(SimpleImputer(strategy='mean'), SVR())

# Entrenar el modelo
modelo.fit(X_train_entrenamiento, y_train_entrenamiento)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Error Cuadrático Medio en el conjunto de evaluación: 1.9866483239291616e+16
Error Porcentual Absoluto Medio en el conjunto de evaluación: 93.01%
[15848429.13, 15848429.76, 15848429.13, 15848494.27, 15848444.92, 15848429.13, 15848429.13, 15848861.82, 15848633.57, 15849078.24, 15848444.92, 15848429.13, 15848473.95, 15848435.09, 15848692.04, 15848429.13, 15848953.7, 15849037.5, 15848429.13, 15848873.03, 15848861.82, 15848692.04, 15848429.13, 15848811.95, 15848429.13, 15848939.32, 15848473.95, 15849051.75, 15848432.8, 15848939.32, 15848444.92, 15848446.66, 15848429.13, 15848431.31, 15848429.13, 15848479.39, 15848658.32, 15848535.22, 15848571.23, 15848429.13, 15848563.6, 15848429.13, 15848473.95, 15848484.98, 15848927.06, 15848429.13, 15848429.13, 15848641.76, 15848429.13, 15848458.65, 15848429.13, 15848692.04, 15848429.13, 15848429.13, 15848924.53, 15848429.13, 15849102.95, 15848873.03, 15848429.13, 15849102.95, 15848593.77, 15848429.13, 15848658.32, 15849149.88, 15848692.04, 15849006.74, 

In [63]:
# Realizar predicciones en el conjunto de evaluación
predicciones_evaluacion = modelo.predict(X_train_evaluacion)

# Calcular el Error Cuadrático Medio (MSE) y el Error Porcentual Absoluto Medio (MAPE)
mse = mean_squared_error(y_train_evaluacion, predicciones_evaluacion)
mape = mean_absolute_error(y_train_evaluacion, predicciones_evaluacion) / y_train_evaluacion.mean() * 100

print(f'Error Cuadrático Medio en el conjunto de evaluación: {mse}')
print(f'Error Porcentual Absoluto Medio en el conjunto de evaluación: {mape:.2f}%')

# Imprimir algunas predicciones redondeadas (opcional)
print([round(prediccion, 2) for prediccion in predicciones_test])

Error Cuadrático Medio en el conjunto de evaluación: 1.9866483239291616e+16
Error Porcentual Absoluto Medio en el conjunto de evaluación: 93.01%
[15848429.13, 15848429.76, 15848429.13, 15848494.27, 15848444.92, 15848429.13, 15848429.13, 15848861.82, 15848633.57, 15849078.24, 15848444.92, 15848429.13, 15848473.95, 15848435.09, 15848692.04, 15848429.13, 15848953.7, 15849037.5, 15848429.13, 15848873.03, 15848861.82, 15848692.04, 15848429.13, 15848811.95, 15848429.13, 15848939.32, 15848473.95, 15849051.75, 15848432.8, 15848939.32, 15848444.92, 15848446.66, 15848429.13, 15848431.31, 15848429.13, 15848479.39, 15848658.32, 15848535.22, 15848571.23, 15848429.13, 15848563.6, 15848429.13, 15848473.95, 15848484.98, 15848927.06, 15848429.13, 15848429.13, 15848641.76, 15848429.13, 15848458.65, 15848429.13, 15848692.04, 15848429.13, 15848429.13, 15848924.53, 15848429.13, 15849102.95, 15848873.03, 15848429.13, 15849102.95, 15848593.77, 15848429.13, 15848658.32, 15849149.88, 15848692.04, 15849006.74, 

## conclusiones

De la presente evaluación es posible concluir que el modelo de Support Vector Machine (SVM) con el conjunto de datos proporcionado parece tener un rendimiento significativamente inferior en compración al modelo de Random Forest. Existe un multiplicidad de factores que pueden contribuir a esta discrepancia, uno de estos radica en la sensibilidad a la escala de las características en SVM, del mismo modo, la necesidad de ajustar adecuadamente los hiperparámetros y el manejo de valores nulos. Podríamos afirmar entonces que en comparación con Random Forest, que puede ser más robusto y menos sensible a la escala, es recomendable centrarse en ajustar y mejorar el modelo de Random Forest, considerando entonces la exploración de diferentes configuraciones de hiperparámetros y asegurándose de que los datos se preparen adecuadamente previo a entrenar el modelo.
